In [204]:
from pypdf import PdfReader


In [205]:
reader = PdfReader("./documents/JUKNIS SPMB JATIM 2025_sign/JUKNIS SPMB JATIM 2025_sign.pdf")

full_text = ""
for page in reader.pages:
    full_text += page.extract_text() + "\n"

print(full_text)

Juknis SPMB Jatim Tahun Ajaran 2025/2026  i 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Nomor : 100.3.6/1425/101.7.1/2025 
Juknis SPMB Jatim Tahun Ajaran 2025/2026  ii 
 
 
 
 
 
 
 
 
Kepala Dinas Pendidikan Provinsi Jawa Timur 
 
Sesuai dengan Undang-Undang Dasar Negara Republik Indonesia Tahun 
1945 bahwa setiap warga negara berhak mendapat pendidikan yang 
bermutu dan berkeadilan. Peningkatan dan pemerataan mutu pendidikan 
menjadi tantangan utama dalam pembangunan pendidikan di Jawa Timur. 
Pemerintah Jawa Timur telah melakukan banyak hal untuk memperbaiki 
dan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, 
salah satunya dengan perbaikan kebijakan seleksi masuk Satuan 
Pendidikan Negeri melalui dikeluarkannya  Petunjuk Teknis tentang 
Pelaksanaan Sistem Penerimaan Murid Baru (SPMB) pada Satuan 
Pendidikan Sekolah Menengah Atas  (SMA), Satuan Pendidikan Sekolah 
Menengah Kejuruan (SMK), dan Satuan Pendidikan  Sekolah Luar Biasa 
(SLB) Tahun Ajaran 2025

In [207]:
from nltk import sent_tokenize, word_tokenize
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import unicodedata

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


class DocumentCleaner:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('indonesian'))

    def normalize_unicode(self, text):
        """Normalize Unicode characters to their closest ASCII representation"""
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    def remove_headers_footers(self, text):
        """Enhanced header and footer removal"""
        header_patterns = [
            r'^.*(?:HEADER:).*$',  # Explicit headers
            r'^\s*\d+\s*of\s*\d+\s*$',  # Page numbers in format "1 of 10"
            r'^.*(?:Last Updated|Last Modified):.*$',
            r'^\s*confidential\s*$',  # Confidential markings
        ]
        footer_patterns = [
            r'^.*(?:FOOTER:).*$',  # Explicit footers
            r'^\s*-\s*\d+\s*-\s*$',  # Page numbers in format "- 1 -"
            r'^.*(?:Copyright|All rights reserved).*$',
        ]

        # Process the text line by line
        lines = text.split('\n')
        cleaned_lines = []

        # Skip first few lines if they match header patterns
        start_idx = 0
        for i, line in enumerate(lines[:3]):  # Check first 3 lines only
            is_header = any(re.match(pattern, line, re.IGNORECASE) for pattern in header_patterns)
            if not is_header:
                start_idx = i
                break

        # Skip last few lines if they match footer patterns
        end_idx = len(lines)
        for i in range(len(lines) - 1, max(len(lines) - 3, start_idx), -1):  # Check last 3 lines only
            is_footer = any(re.match(pattern, line, re.IGNORECASE) for pattern in footer_patterns)
            if not is_footer:
                end_idx = i + 1
                break

        # Keep the content between start_idx and end_idx
        cleaned_lines = lines[start_idx:end_idx]

        return '\n'.join(cleaned_lines).strip()

    def remove_special_characters(self, text):
        """Enhanced special character removal"""
        # Keep essential punctuation but remove other special characters
        allowed_chars = set(punctuation) - {'*', '/', '\\', '|', '@', '#', '$', '%', '^', '&'}
        text = ''.join(char if char.isalnum() or char.isspace() or char in allowed_chars else ' ' for char in text)
        return re.sub(r'\s+', ' ', text).strip()

    def normalize_whitespace(self, text):
        """Normalize all types of whitespace"""
        # Replace all types of whitespace with a single space
        text = re.sub(r'[\s\xa0\u200b\u3000]+', ' ', text)
        # Normalize line breaks
        text = re.sub(r'\n\s*\n', '\n\n', text)
        return text.strip()

    def remove_redundant_content(self, text):
        """Remove redundant content including repeated phrases and boilerplate text"""
        # Remove repeated words
        text = re.sub(r'\b(\w+)(\s+\1\b)+', r'\1', text)

        # Remove repeated substring
        text = re.sub(r'\.{2,}', '.', text)

        # Remove common boilerplate phrases
        boilerplate = [
            'untuk informasi lebih lanjut',
            'sebagaimana dijelaskan di atas',
            'dengan hormat',
            'perhatian:',
            'catatan:',
            'nb:',
            'note:'
        ]
        for phrase in boilerplate:
            text = re.sub(rf'{phrase}\s*', '', text, flags=re.IGNORECASE)
        return text.strip()

    def clean_numbers_and_dates(self, text):
        """Standardize number and date formats"""
        # Standardize date formats (DD-MM-YYYY)
        text = re.sub(r'(\d{1,2})/(\d{1,2})/(\d{4})', r'\1-\2-\3', text)
        # Remove isolated numbers that aren't part of meaningful content
        text = re.sub(r'\s+\d+\s+', ' ', text)
        return text.strip()

    def clean_document(self, text):
        """Main cleaning pipeline"""
        # Initial normalization
        text = self.normalize_unicode(text)
        text = self.normalize_whitespace(text)

        # Structure cleaning
        text = self.remove_headers_footers(text)
        text = self.clean_numbers_and_dates(text)

        # # Content cleaning
        text = self.remove_special_characters(text)
        text = self.remove_redundant_content(text)

        # Linguistic processing
        sentences = sent_tokenize(text)
        cleaned_sentences = []

        for sentence in sentences:
            # Tokenize and remove stopwords
            words = word_tokenize(sentence)
            words = [word.lower() for word in words if word.lower() not in self.stop_words]

            # Lemmatize
            words = [self.lemmatizer.lemmatize(word) for word in words]

            # Reconstruct sentence
            if words:
                cleaned_sentences.append(' '.join(words))

        return ' '.join(sentences)


# Update the preprocessing part of your code
cleaner = DocumentCleaner()
clean_text = cleaner.clean_document(full_text)

# Continue with your existing chunking and RAG pipeline...

[nltk_data] Downloading package punkt to /home/robby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/robby/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/robby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/robby/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [208]:

print(clean_text)


Juknis SPMB Jatim Tahun Ajaran 2025 2026 i Nomor : 100.3.6 1425 101.7.1 2025 Juknis SPMB Jatim Tahun Ajaran 2025 2026 ii Kepala Dinas Pendidikan Provinsi Jawa Timur Sesuai dengan Undang-Undang Dasar Negara Republik Indonesia Tahun bahwa setiap warga negara berhak mendapat pendidikan yang bermutu dan berkeadilan. Peningkatan dan pemerataan mutu pendidikan menjadi tantangan utama dalam pembangunan pendidikan di Jawa Timur. Pemerintah Jawa Timur telah melakukan banyak hal untuk memperbaiki dan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, salah satunya dengan perbaikan kebijakan seleksi masuk Satuan Pendidikan Negeri melalui dikeluarkannya Petunjuk Teknis tentang Pelaksanaan Sistem Penerimaan Murid Baru (SPMB) pada Satuan Pendidikan Sekolah Menengah Atas (SMA), Satuan Pendidikan Sekolah Menengah Kejuruan (SMK), dan Satuan Pendidikan Sekolah Luar Biasa (SLB) Tahun Ajaran 2025 2026. Untuk mempercepat pencapaian sasaran pembangunan pendidikan yang bermutu dan berkeadilan ter

In [210]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "]
)

In [211]:
new_chunks = splitter.split_text(clean_text)
print(len(new_chunks))

241


In [212]:
for i, chunk in enumerate(new_chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n{'-' * 40}")
    print(f"Length: {len(chunk)}")

Chunk 1:
Juknis SPMB Jatim Tahun Ajaran 2025 2026 i Nomor : 100.3.6 1425 101.7.1 2025 Juknis SPMB Jatim Tahun Ajaran 2025 2026 ii Kepala Dinas Pendidikan Provinsi Jawa Timur Sesuai dengan Undang-Undang Dasar Negara Republik Indonesia Tahun bahwa setiap warga negara berhak mendapat pendidikan yang bermutu dan berkeadilan. Peningkatan dan pemerataan mutu pendidikan menjadi tantangan utama dalam pembangunan pendidikan di Jawa Timur. Pemerintah Jawa Timur telah melakukan banyak hal untuk memperbaiki dan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, salah satunya dengan perbaikan kebijakan seleksi masuk Satuan Pendidikan Negeri melalui dikeluarkannya Petunjuk Teknis tentang Pelaksanaan Sistem Penerimaan Murid Baru (SPMB) pada Satuan Pendidikan Sekolah Menengah Atas (SMA), Satuan Pendidikan Sekolah Menengah Kejuruan (SMK), dan Satuan Pendidikan Sekolah Luar Biasa (SLB) Tahun Ajaran 2025 2026
----------------------------------------
Length: 906
Chunk 2:
. Untuk mempercepat pe

In [213]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

token_splitter = SentenceTransformersTokenTextSplitter(
    chunk_overlap=0, tokens_per_chunk=384)

token_split_texts = []

for text in new_chunks:
    token_split_texts.extend(token_splitter.split_text(text))

print(len(token_split_texts))


254


In [214]:
print(token_split_texts[0])

juknis spmb jatim tahun ajaran 2025 2026 i nomor : 100. 3. 6 1425 101. 7. 1 2025 juknis spmb jatim tahun ajaran 2025 2026 ii kepala dinas pendidikan provinsi jawa timur sesuai dengan undang - undang dasar negara republik indonesia tahun bahwa setiap warga negara berhak mendapat pendidikan yang bermutu dan berkeadilan. peningkatan dan pemerataan mutu pendidikan menjadi tantangan utama dalam pembangunan pendidikan di jawa timur. pemerintah jawa timur telah melakukan banyak hal untuk memperbaiki dan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, salah satunya dengan perbaikan kebijakan seleksi masuk satuan pendidikan negeri melalui dikeluarkannya petunjuk teknis tentang pelaksanaan sistem penerimaan murid baru ( spmb ) pada satuan pendidikan sekolah menengah atas ( sma ), satuan pendidikan sekolah menengah kejuruan ( smk ), dan satuan pendidikan sekolah luar biasa ( slb ) tahun ajaran 2025 2026


In [216]:
import chromadb

client = chromadb.HttpClient(host="localhost", port=8000)

# Test connection
client.heartbeat()
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    # Optional: Specify the embedding dimension if known
)

chroma_collection = client.create_collection("mahasiswa", embedding_function=embedding_function)

Exception: {"error":"ChromaError","message":"Collection [mahasiswa] already exists"} (trace ID: 00000000000000000000000000000000)

In [217]:
ids = [str(i) for i in range(len(token_split_texts))]

metadata = [{"text": text} for text in token_split_texts]

# Insert the data into the collection
chroma_collection.add(
    documents=token_split_texts,
    metadatas=metadata,
    ids=ids
)

In [6]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()


def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": (
                "Kamu adalah asisten yang ahli dan membantu dalam mencari informasi dari buku petunjuk teknis. "
                "Pengguna akan mengajukan pertanyaan berdasarkan informasi yang diambil dari buku petunjuk teknis. "
                "Jawablah pertanyaan pengguna hanya berdasarkan informasi yang diberikan."
            )
        },
        {
            "role": "user",
            "content": f"Pertanyaan: {query} \n\nInformasi: {retrieved_documents}"
        }
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content


In [7]:
query = "Apa itu SPMB?"


In [8]:
def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "Anda adalah asisten yang sangat membantu dalam mencari sesuatu di dokumen peraturan petunjuk teknis."
                       "Sarankan hingga lima pertanyaan tambahan yang terkait untuk membantu mereka menemukan informasi yang mereka butuhkan, untuk pertanyaan yang diberikan. "
                       "Sarankan hanya pertanyaan singkat tanpa kalimat majemuk. Sarankan berbagai pertanyaan yang mencakup berbagai aspek topik."
                       "Pastikan pertanyaan-pertanyaan tersebut lengkap, dan berhubungan dengan pertanyaan awal."
                       "Keluarkan satu pertanyaan per baris. Jangan beri nomor dan dash pada pertanyaan."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

In [9]:
augmented_queries = augment_multiple_query(query)

queries = [query] + augmented_queries
for q in queries:
    print(q)


Apa itu SPMB?
- Bagaimana proses pendaftaran peserta SPMB?
- Apa saja persyaratan untuk mengikuti SPMB?
- Bagaimana cara mengakses informasi mengenai jadwal pelaksanaan SPMB?
- Apa yang perlu dipersiapkan untuk mengikuti ujian SPMB?
- Bagaimana mekanisme pelaksanaan ujian SPMB?


In [10]:
results = chroma_collection.query(
    query_texts=queries,
    n_results=5,
    include=['documents', 'embeddings']
)

retrieved_documents = results["documents"]

unique_documents = set()

for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

for i, documents in enumerate(retrieved_documents):
    print(f"Documents for augmented query {i + 1}:")
    for document in documents:
        print(f"{i + 1}. {document}")
    print()


NameError: name 'chroma_collection' is not defined

In [287]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [306]:
pairs = []
for doc in unique_documents:
    pairs.append([query, doc])

print(pairs)

[['Berikan saya jadwal Tahap 1 Dalam SPMB Jatim 2025', '. jalur prestasi adalah jalur dalam penerimaan murid baru yang diperuntukkan bagi calon murid yang memiliki prestasi di bidang akademik dan atau nonakademik. jalur mutasi adalah jalur dalam penerimaan murid baru yang diperuntukkan bagi calon murid yang berpindah domisili karena perpindahan tugas dari orang tua wali dan bagi anak guru yang mendaftar di satuan pendidikan tempat orang tua mengajar. mekanisme yang digunakan pada spmb tahun ajaran 2025 2026 adalah dengan moda dalam jaringan ( daring ) secara penuh kecuali beberapa satuan pendidikan dilaksanakan secara luar jaringan ( luring )'], ['Berikan saya jadwal Tahap 1 Dalam SPMB Jatim 2025', '. 1. 01. 0. 00. 0. 00. 01. 0000 001 2025 tanggal januari 2025. kesembilan : keputusan ini mulai berlaku sejak tanggal ditetapkan sampai dengan berakhirnya kegiatan dengan ketentuan apabila terdapat kekeliruan akan diperbaiki sebagaimana mestinya. ditetapkan : di surabaya tanggal : maret kep

In [307]:
scores = cross_encoder.predict(pairs)


In [5]:
print("Scores:")
for score in scores:
    print(score)

Scores:


NameError: name 'scores' is not defined

In [309]:
import numpy as np

print("New Ordering:")
for o in np.argsort(scores)[::-1]: \
        print(o)

New Ordering:
9
13
3
12
11
5
4
8
2
0
10
1
7
6


In [310]:
# Filter Top 3 Answer from scorring
top_3_docs = [pairs[i][1] for i in np.argsort(scores)[::-1][:3]]
print(top_3_docs)

['juknis spmb jatim tahun ajaran 2025 2026 i nomor : 100. 3. 6 1425 101. 7. 1 2025 juknis spmb jatim tahun ajaran 2025 2026 ii kepala dinas pendidikan provinsi jawa timur sesuai dengan undang - undang dasar negara republik indonesia tahun bahwa setiap warga negara berhak mendapat pendidikan yang bermutu dan berkeadilan. peningkatan dan pemerataan mutu pendidikan menjadi tantangan utama dalam pembangunan pendidikan di jawa timur. pemerintah jawa timur telah melakukan banyak hal untuk memperbaiki dan meningkatkan kualitas pendidikan yang bermutu dan berkeadilan, salah satunya dengan perbaikan kebijakan seleksi masuk satuan pendidikan negeri melalui dikeluarkannya petunjuk teknis tentang pelaksanaan sistem penerimaan murid baru ( spmb ) pada satuan pendidikan sekolah menengah atas ( sma ), satuan pendidikan sekolah menengah kejuruan ( smk ), dan satuan pendidikan sekolah luar biasa ( slb ) tahun ajaran 2025 2026', '. juknis spmb jatim tahun ajaran 2025 2026 4. jalur domisili ( sma smk ) a

In [311]:
full_answer = ""
for doc in top_3_docs:
    full_answer += doc + "\n"
full_answer += "\n"
full_answer = rag(query, full_answer)

print("Question: ", query)
print("Answer:")
print(full_answer)

Question:  Berikan saya jadwal Tahap 1 Dalam SPMB Jatim 2025
Answer:
Mohon maaf, informasi yang Anda berikan tidak menyediakan jadwal Tahap 1 dalam SPMB Jatim 2025. Apakah ada informasi lain yang dapat saya bantu?
